In [1]:
import sys
sys.path.insert(0, '../../src')
import feedparser
import pandas as pd

import urllib

from postagem.Util import extract_domain, download_and_move_image, get_noticia_comercio
from lexical_analyzer_package import midia_lexical
from midia_postagem import midia_post
from Model.News import News
from Database import midia_table

from newsplease import NewsPlease
from bs4 import BeautifulSoup
import requests

import datetime
import util_midia

In [2]:
link = 'https://canaltech.com.br/'

In [7]:
link_site = 'https://canaltech.com.br'

In [8]:
req = requests.get(link)

In [9]:
noticias = BeautifulSoup(req.text, "html.parser").find_all('div', class_='item row')

In [18]:
for noticia in noticias:
    print(noticia.find_all('a', href=True)[0]['href'])
    ref_link = noticia.find_all('a', href=True)[0]['href']
    print(link_site+ref_link)
    if (link_site not in ref_link):
        util_midia.social_news_from_link(link_site+ref_link)
        print(link_site+ref_link)
    elif(ref_link == ref_link):
        continue
    else:
        util_midia.social_news_from_link(ref_link)
        print(ref_link)

/e-commerce/correios-estao-encaminhando-eletronicos-importados-para-homologacao-pela-anatel-125690/
https://canaltech.com.br/e-commerce/correios-estao-encaminhando-eletronicos-importados-para-homologacao-pela-anatel-125690/
HTTP Error 403: Forbidden
['Correios estão encaminhando eletrônicos importados para homologação pela Anatel']
news_in_db: False
 -- no categories -- 
https://canaltech.com.br/e-commerce/correios-estao-encaminhando-eletronicos-importados-para-homologacao-pela-anatel-125690/
/e-commerce/correios-estao-encaminhando-eletronicos-importados-para-homologacao-pela-anatel-125690/
https://canaltech.com.br/e-commerce/correios-estao-encaminhando-eletronicos-importados-para-homologacao-pela-anatel-125690/
HTTP Error 403: Forbidden
['Correios estão encaminhando eletrônicos importados para homologação pela Anatel']
news_in_db: False
 -- no categories -- 
https://canaltech.com.br/e-commerce/correios-estao-encaminhando-eletronicos-importados-para-homologacao-pela-anatel-125690/
/mer

In [ ]:
noticias[0]